In [ ]:
import pandas as pd


# Import the necessary libraries
import os
from openai import OpenAI

client = OpenAI(api_key = api2)

def prompt(prompt, m, s, t, results, temp=1):

    #results = {'prompt':[], 'response': [], 'model': [], 'temperature': [], 'seed':[], 'NEM': []}
    try:
        response = client.chat.completions.create(
            model = m,
            seed = s,
            temperature = temp,
            messages=[
                {"role": "system", "content": "You are ChatGPT."},
                {"role": "user", "content": prompt}
            ],
            max_completion_tokens=100000,  # Adjusted to a reasonable length for 10 sentences
        )
    except openai.OpenAIError as e:
        print(f"OpenAI API error: {e}")

    a = response.choices[0].message.content.strip()

    results['prompt'].append(prompt)
    results['temperature'].append(temp)
    results['response'].append(a)
    results['model'].append(m)
    results['seed'].append(s)
    results['text'].append(t)


    return results



In [ ]:

import pandas as pd

languages = ['english']
seeds = [0,5]
#a.head()

pr = """Can you classify whether the phrase between [PET_BOUNDARY] delimiters in the text is being used in a euphemistic manner or not? Do not include an intro or conclusion in your answer. Just output the number 1 if its euphemistic, 0 if not.

Here are some texts and their corresponding outputs::
Text: 'perhaps as many as, more people have been [PET_BOUNDARY]laid off[PET_BOUNDARY] since march, said one former employee who was let go in april.', output: 1.
Text: 'you may have recently heard about the option of life settlements, where you sell your life insurance policy to a third party, and be considering this option in order to fund assisted living. the buyer of the policy takes responsibility for the premiums, and then receives the benefit when you [PET_BOUNDARY]pass away[PET_BOUNDARY].', output: 1.
Text: 'They're great on paper, but when it comes time to get [PET_BOUNDARY]down there[PET_BOUNDARY] in the trenches, they get all squirrelly.', output: 0.
Text: 'If I have roughly $600 dollars invested in my team (minimum, per season), I am [PET_BOUNDARY]expecting[PET_BOUNDARY] that the millionaires who get paid a heaping sum will perform their duties to a reasonable expectation.', output: 0.

This is the text: '
"""
for x in languages:
    for y in range(seeds[0],seeds[1]):


        a = pd.read_csv(x+' euphs 2025-09-17/test_0_'+x+'.csv')
        results = {'prompt':[], 'text': [], 'response': [], 'model': [], 'temperature': [], 'seed':[]}

        for i in a['TEXT']:
            prompt(pr + i + "'", 'gpt-5-nano', y, i, results, 1)
        results = pd.DataFrame(results)
        results['true_value'] = a['LABEL']
        results.to_csv('fewshot_prompt2_'+x+'_'+str(y)+'.csv')
